# Generating topics for venue tips in London
* prepare tips data
* preprocessing tips
* generating topics
* evaluation

### 1.prepare tips data
We collect tips from Foursquare. 

There are around 11000 places and about 1700 places have more 200 tips.
All of the tips(English tips) were collected previously. 

In [ ]:
# import useful libs
import graphlab as gl
import pandas as pd
import json
import re
import pyLDAvis 
import pyLDAvis.graphlab

In [65]:
# load the pre-processed data
sf = gl.SFrame.read_json('london_venue.json')
#sf.head()

Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\Recommender System\london_venue.json

Parsing completed. Parsed 1 lines in 0.34851 secs.

### 2. preprocessing
* put all tips for one venue together, store them in a new column 'all_tips'

In [69]:
# put all tips together
def get_feature(row):
    feature=''
    if row['tips']!=None:
        for t in row['tips']:
            feature += t
    
    tags = 5*row['tags']
    for tag in tags:
        feature += tag+' '
    if row['description']!=None:
        feature += row['description']
        feature += ' '
        feature += row['description']
    return feature
sf['feature'] = sf.apply(lambda x: get_feature(x))

   * Text cleaning
   * Lemmatization

In [70]:
# get words, dropping punctuations etc.
sf['feature'] = sf['feature'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))
docs = gl.text_analytics.tokenize(sf['feature'])

In [71]:
# lemmatization
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
docs_lemmatized = docs.apply(lambda x: [wordnet_lemmatizer.lemmatize(w) for w in x])
#docs_lemmatized[0]

* Tokenization
* Bag-of-words representation
* Stop words and less frequent words removal

In [57]:
# Bag-of-words
docs_cleaned = gl.text_analytics.count_words(docs_lemmatized)
# Remove stop words
docs_cleaned = docs_cleaned.dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)
# Remove less freq words
docs_cleaned = docs_cleaned.dict_trim_by_values(2)

###  3. Gnerate topics
* remove docs which has less than 3 keywords
* create a model
* check and evaluate

In [104]:
# remove docs which has less than 3 keywords
ix = docs_cleaned.apply(lambda x:len(x.keys())>2)
docs_new = docs_cleaned[ix]

In [105]:
# Show how many docs have been removed
print 1 - 1.0*len(docs_new)/len(docs_cleaned)

0.559368737475


In [106]:
# create a topic model 
#associations = gl.SFrame({'word':['local','authentic', 'london', 'good', 'lunch','delicious','amazing','lover','walk'],
#                                'topic': [0, 0, 0, 1, 1,2,2,3,3]})

topic_model = gl.topic_model.create(docs_new,num_topics=20, num_iterations=200)

Learning a topic model

Number of documents      5277

Vocabulary size     11267

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 170       | 4.62s         | 4.58414e+006   | 0               |

| 180       | 4.87s         | 9.16799e+006   | 0               |

| 190       | 5.17s         | 4.58502e+006   | 0               |

| 200       | 5.45s         | 4.58487e+006   | 0               |

+-----------+---------------+----------------+-----------------+

In [109]:
for i in range(20):
    print 'topic ',i,topic_model.get_topics(num_words=5,output_type='topic_words')[i]['words']

topic  0 ['coffee', 'staff', 'place', 'nice', 'friendly']
topic  1 ['bar', 'cocktail', 'great', 'music', 'place']
topic  2 ['art', 'free', 'museum', 'place', 'exhibition']
topic  3 ['food', 'great', 'wine', 'restaurant', 'menu']
topic  4 ['station', 'train', 'time', 'free', 'don']
topic  5 ['food', 'good', 'sushi', 'great', 'chicken']
topic  6 ['food', 'good', 'service', 'wa', 'staff']
topic  7 ['burger', 'great', 'good', 'chicken', 'wa']
topic  8 ['park', 'place', 'great', 'london', 'garden']
topic  9 ['london', 'don', 'worth', 'good', 'time']
topic  10 ['market', 'store', 'shop', 'street', 'gym']
topic  11 ['food', 'pizza', 'restaurant', 'italian', 'good']
topic  12 ['palace', 'place', 'queen', 'wa', 'buckingham']
topic  13 ['london', 'office', 'uk', 'event', 'offer']
topic  14 ['great', 'good', 'breakfast', 'food', 'place']
topic  15 ['design', 'book', 'medium', 'student', 'digital']
topic  16 ['great', 'theatre', 'wa', 'cinema', 'seat']
topic  17 ['view', 'london', 'bridge', 'tower

In [ ]:
# These are the 5th topic words
print topic_model.get_topics(num_words=10,output_type='topic_words')[11]

In [ ]:
docs_in_topic_5 = docs_new[topic_model.predict(docs_new)==11]

In [ ]:
# select venues which are predicted to be topic 5
sf_new = sf[ix]
venue_in_topic5 = sf_new[topic_model.predict(docs_new)==11]

In [ ]:
venue_in_topic5['category','name','tips'][21]

In [110]:
#Automatic Rendering of Visualisation
pyLDAvis.enable_notebook()
pyLDAvis.graphlab.prepare(topic_model, docs_new)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
9      8.197592        1       1 -0.056895 -0.191128
14     6.539526        1       2  0.205498 -0.005070
3      6.121186        1       3  0.227488  0.067383
5      6.041709        1       4  0.232496  0.040245
7      6.006437        1       5  0.131171  0.030527
2      5.390577        1       6 -0.170906 -0.062858
6      5.255693        1       7  0.212324 -0.062859
18     5.079938        1       8 -0.002116 -0.114566
1      5.045038        1       9  0.025442  0.087242
8      4.998065        1      10 -0.116858 -0.132903
19     4.916893        1      11  0.164089  0.007436
0      4.883792        1      12 -0.020771 -0.143279
4      4.697822        1      13 -0.206111 -0.094869
11     4.427472        1      14  0.222654  0.010760
17     4.290750        1      15 -0.204803 -0.136926
10     4.271340        1      16 -0.140417  0.117990
12     4.204813        1      17 -0.088415  0.140044
16     3.934070        1      18 -0.008432  0.048113
13     2.864075        1      19 -0.213655  0.139184
15     2.833212        1      20 -0.191783  0.255533, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
3014   Default   5698.000000         coffee   5698.000000  30.0000  30.0000
4408   Default  14053.000000           food  14053.000000  29.0000  29.0000
8636   Default   9870.000000         london   9870.000000  28.0000  28.0000
2162   Default  16686.000000          great  16686.000000  27.0000  27.0000
7280   Default  13577.000000           good  13577.000000  26.0000  26.0000
10967  Default   3215.000000           view   3215.000000  25.0000  25.0000
4286   Default   2975.000000            pub   2975.000000  24.0000  24.0000
8914   Default   2796.000000           beer   2796.000000  23.0000  23.0000
274    Default   7052.000000        service   7052.000000  22.0000  22.0000
10196  Default   2740.000000          hotel   2740.000000  21.0000  21.0000
7391   Default   3089.000000           room   3089.000000  20.0000  20.0000
5947   Default   4168.000000            bar   4168.000000  19.0000  19.0000
9433   Default  10198.000000          place  10198.000000  18.0000  18.0000
3226   Default   7539.000000             wa   7539.000000  17.0000  17.0000
8117   Default   4644.000000          staff   4644.000000  16.0000  16.0000
7129   Default   2565.000000         burger   2565.000000  15.0000  15.0000
10831  Default   2287.000000       cocktail   2287.000000  14.0000  14.0000
6940   Default   6486.000000           nice   6486.000000  13.0000  13.0000
6545   Default   1876.000000          pizza   1876.000000  12.0000  12.0000
11051  Default   3930.000000     restaurant   3930.000000  11.0000  11.0000
3862   Default   2443.000000      breakfast   2443.000000  10.0000  10.0000
165    Default   2060.000000           wine   2060.000000   9.0000   9.0000
7375   Default   1828.000000           park   1828.000000   8.0000   8.0000
8710   Default   2972.000000           free   2972.000000   7.0000   7.0000
5731   Default   5040.000000        amazing   5040.000000   6.0000   6.0000
99     Default   1718.000000          music   1718.000000   5.0000   5.0000
4500   Default   1487.000000        station   1487.000000   4.0000   4.0000
10934  Default   1913.000000        chicken   1913.000000   3.0000   3.0000
2613   Default   2292.000000          drink   2292.000000   2.0000   2.0000
4935   Default   2609.000000       friendly   2609.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
1626   Topic20     65.734054       research     67.568523   3.5362  -5.5037
5295   Topic20     62.704835       computer     64.539303   3.5349  -5.5508
11003  Topic20     52.607438            job     54.441907   3.5295  -5.7264
7022   Topic20     52.607438      bookstore     54.4419